# OpenCV을 이용한 신발 인식프로그램

## 프로젝트 목표 및 이 프로젝트의 필요성

패션 관련 유튜브를 볼 때 흔히 "이 옷 어디 거에요?"하고 정보를 물어보는 댓글들을 흔히 볼 수 있습니다.<br>
![youtube](report_photos/youtube.png)<br>
![youtubecomment](report_photos/comments.png)<br>
(2020.10.14일 기준 13시간이 지난 영상에 130여 개 정도의 댓글에서 종종 찾아볼 수 있었습니다.)<br>
유튜버 입장에서는 댓글이 영상마다 많이 달리기 때문에 일일이 답변을 하기 힘들 것이고, 댓글을 쓴 시청자 입장에서는 원하는 대답을 얻기 어려울 것입니다.<br>
그렇기 때문에 이미지를 분석해서 그 제품이 무엇인지 알려준다면 이러한 문제를 적절하게 해결할 수 있다고 생각을 하였습니다.<br>
다양한 품목들 중 먼저 신발을 대상으로 선정하여 적절하게 구별해낼 수 있도록 구현하여 다른 품목들도 확장해나갈 수 있는 발판을 세우는 것이 이번 프로젝트의 목표입니다.<br>
초기 계획에서는 상품을 찾고 그 상품에 대한 url을 연결해주는 쪽으로 구상하였습니다.<br>
하지만 이용자마다 선호하는 쇼핑몰이 다를 수 있고, 많은 쇼핑몰에 같은 상품이 존재하기 때문에  그 상품에 대한 적절한 사이트를 추천하는 것은비효율적이라고 판단하였습니다.<br>
따라서 url 연결을 하는 부분을 제외하고 사진을 보다 편리하게 선택하면서 상품을 찾을 수 있는 것에 집중하여 GUI로 프로그램을 구성하기로 결정하였습니다.<br>

## 프로토타입

![PROTOTYPE](report_photos/prototype.png)

## 데이터 획득(크롤링)

무신사 스토어 2020.10.18일 기준 스니커즈 부분 주간 랭킹 1,2,3 위인 척 70 클래식 블랙 162058Cb, BW 독일군 스니커즈 화이트 BZ0579, 에어 조던1 하이 OG 디올 리미티드 에디션으로 프로젝트를 진행하였습니다.<br>
![ranking](report_photos/10.18_ranking.png)

-결과<br>
<img src="report_photos/crawling_result1.png" align="left">
![crawling_reulst](report_photos/crawling_result2.png)
<br><br>
결과 내에서 적절하지 못한 사진은 수작업으로 제외하였습니다.(신발 안쪽, 바닥 사진, 신발 박스 사진, 같은 브랜드 다른 신발 등)<br><br>

소스코드: photographic_crawing.py

## 이미지 데이터 만들기

YOLO모델로 학습시키기 전에 적절한 형태로 데이터를 가공하였습니다.<br>
labelimg 툴(https://github.com/tzutalin/labelImg) 을 이용하여 선별한 사진들에서의 신발의 위치를 나타낸 txt파일을 얻었습니다.<br>
![labelimg_result](report_photos/train_set(labelImg).png)<br>
<img src="report_photos/labelImg_txt.png" align="left"> <br><br><br><br>
txt 파일은 class, centerX, centerY, width, height 으로 구성되어 있습니다.<br>
Google Drive의 Darknet 폴더의 custom에 이미지 데이터를 넣었놓았습니다.<br>

##  학습데이터 만들기

#### my_class.name 생성
labelImg를 수행하여 함께 생성되었던 class.txt 파일(각 class의 name이 적혀있는 txt파일)을 my_class.name 파일로 수정하였습니다.<br>
<img src="report_photos/my_classname.png" align="left"><br><br><br><br><br><br><br>
#### my_yolo.cfg 생성
darknet의 cfg 파일 중 yolov3.cfg의 복사본을 만들어 수정하였습니다.<br>

[net] 에서는 한 번에 64개의 이미지를 읽어오도록 하고 최소 16개를 읽어올 수 있도록 의 batch를 64로 수정하고 subdivision은 16로 수정하였습니다.<br>
max_batches 값은 class가 3개인 것을 가만하여 (calss) * 2000해서 6000개로 지정하였고, steps는 max_batches의 80%와 90%로 값인 4800, 5400으로 바꾸었습니다.<br>
[yolo]에서 classes를 3으로 바꾸었습니다.<br>
[yolo]위의 [convolutional]에서 filters 값은 (classes + coordinates + 1) * masks로 계산하여 (3+5)*3 = 24로 설정하였습니다.<br>

#### my_text.txt, my_train.txt 생성
각 .jpg 파일의 위치를 나타내는 txt파일을 생성하였습니다. 8대2의 비율로 shuffle을 활용하여 랜덤하게 train data와 test data로 나누었습니다.<br> Google Drive를 이용해서 COLAB을 사용할 계획이므로 Google Drive의 경로로 사진을 지정하였습니다.
-결과<br>
![labelimg_result](report_photos/train&test_txt.png)
소스코드: train&test_txtfile.py

#### my_obj.data 생성
class = 3<br>
train = /content/gdrive/My\ Drive/darknet/custom/my_train.txt<br>
valid = /content/gdrive/My\ Drive/darknet/custom/my_test.txt<br>
names = /content/gdrive/My\ Drive/darknet/custom/my_classes.names<br>
backup = backup<br>
위와 같은 구성으로 my_obj.data 파일을 만들었습니다.

## COLAB 개발환경 구축 및 데이터 학습

GPU를 사용하여 performance를 높이기 위해서 COLAB을 이용하여 프로젝트를 진행하였습니다.<br>
개인 Google Drive에 YOLO로 학습을 하기위해 필요한 파일들을 구성해놓은 뒤 COLAB에 연결하였습니다.<br>
COLAB에 darknet을 설치한 뒤 총 6000번의 학습을 진행하였습니다.<br>
<img src="report_photos/5700...png" align="left"><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
5712번까지 학습하던 도중 학습이 멈춰 다음과 같이 5000번째 부터 다시 이어서 학습을 진행하였습니다.(6000번째에서 중단, 6000번째는 final로 저장)<br>
![:(](report_photos/re5000.png)<br>
결과:my_yolo_1000.weights,my_yolo_2000.weights,my_yolo_3000.weights,my_yolo_4000.weights,my_yolo_5000.weights,my_yolo_final.weights<br>
(Google Drive의 darknet/backup 폴더에 복사하여 넣어두었습니다.)<br><br>
Google Drive: https://drive.google.com/drive/folders/1bw7MUi8tJGqyEY5x9c0--_-b_uJD0lKe?usp=sharing<br>
소스코드: openCV_training.ipynb<br>

## 적합한 YOLO 학습 모델 선정

학습하는 과정에서 오버피팅(Overfitting)이 발생했었을 수도 있기때문에 최적의 YOLO 학습 모델을 찾기 위해서 1000개 단위로 저장한 weights파일들의 mean average precision값을 구해보았습니다.<br>
![fit_yolo](report_photos/1000.png)
my_yolo_1000.weights의 경우 mean average precision가 89.07%
![fit_yolo](report_photos/2000.png)
my_yolo_2000.weights의 경우 mean average precision가 96.57%
![fit_yolo](report_photos/3000.png)
my_yolo_3000.weights의 경우 mean average precision가 95.36% 
![fit_yolo](report_photos/4000.png)
my_yolo_4000.weights의 경우 mean average precision가 96.47%
![fit_yolo](report_photos/5000.png)
my_yolo_5000.weights의 경우 mean average precision가 95.59% 
![fit_yolo](report_photos/6000.png)
my_yolo_6000.weights의 경우 mean average precision가 95.27% 라는 결과가 나왔습니다.<br>
my_yolo_2000.weights가 96.57%로 가장 높은 mean average precision을 기록했고, 4000번 학습한 이후에는 오히려 mean average precision이 떨어지는 경향이 나타나 오버피팅이 되었다고 판단하였습니다.<br>
따라서 가장 높은 mean average precision을 기록한 my_yolo_2000.weights 학습 모델을 이용하여 프로그램을 제작하기로 결정하였습니다.<br><br>
소스코드: openCV_training.ipynb<br>
COLAB 링크: https://colab.research.google.com/drive/1D0ePX-D2UpfAd58RZ8tR2DNYkL9ZybdN?usp=sharing<br>
(데이터학습부분도 함께 포함되어 있으며 내용은 openCV_training.ipynb와 같습니다.)

## 신발인식 GUI프로그램 제작

소스코드: shoes recognition program.ipynb

실행 결과
![result](report_photos/result.png)
![result](report_photos/result1.png)
위 사진에서 보듯이 신발인식과 정확도 변경 및 사진 변경이 원활하게 되었습니다.<br>
(테스트에 사용한 사진들은 학습에 사용하지 않은 사진들이며, 프로그램 수요자의 취지에 맞추기 위해 신발만 나타나있는 사진이 아니라 전신이나 하반신까지 나오는 사진을 선택하였습니다.)<br>

## 한계점

1. 신발 자체를 훈련시킨 사진들과 다르게 사진 속 신발의 사이즈가 작아서 그런지 인식률이 많이 떨어졌습니다.<br>
![fit_yolo](report_photos/threshold_point.png)
<br>
2. 크롤링하여 얻은 신발 사진 대부분이 신발 두 쪽을 한 번에 학습시키기에 적절하지 않은 이미지여서 신발 한 쪽씩 훈련시켰더니 각 신발마다 인식률이 달랐고, 신발이 가까우 붙어있는 경우 글씨가 붙어서 잘 알아보기 힘든 경우가 발생하였습니다.<br>
![fit_yolo](report_photos/threshold_point3.png)
<br>
3. 크롤링을 통해 얻은 사진들이 중복되는 경우가 많아서 이로 인해 정확도가 떨어졌을 가능성이 있습니다.<br>


## 참고자료

### 주요 참고자료만 남겼습니다. 추가적으로 구글링을 주로 활용하였습니다.
- github, AlexeyAB/darknet https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects
- Stack Overflow, https://stackoverflow.com/
- 파이썬 표준 라이브러리, "tkinter — Tcl/Tk 파이썬 인터페이스", https://docs.python.org/ko/3/library/tkinter.html
- pyimagesearch,"Object detection with deep learning and OpenCV",https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/
- 크리애플, https://www.creapple.com/home/main
- 어쩐지 오늘은, "openCV - 이미지/ 비디오 읽기", https://zzsza.github.io/data/2018/01/23/opencv-1/
- 봉식이와 캔따개, "Python으로 OpenCV를 사용하여 YOLO Object detection", https://bong-sik.tistory.com/16
- Enough is not enough, "[Object Detection] darknet custom 학습하기", https://eehoeskrap.tistory.com/370 [Enough is not enough]
- Yun Dae Hee, "Python tkinter 강좌", https://076923.github.io/posts/Python-tkinter-32/
- 오세원 블로그, "Theft Chaser : Yolo를 활용한 도둑탐지 - Labeling, Yolo 학습", https://osw51.tistory.com/6
- J_Remind, "(YOLO) 이미지 학습", https://j-remind.tistory.com/64
- DSC Ewha ,"Image labeling 및 Yolo darkflow/darknet관련 프로젝트 분석",https://dsc-ewha.tistory.com/51
- catharsis, "[YOLO - darkflow] YOLO와 Python을 이용한 object detection (2) - image detection" ,https://reyrei.tistory.com/17